# Selected optimization topics

  * Avoid unneeded data copies
    * emplace_back vs push_back
    * Pass and receive by reference      
  * Improve cache locality
    * Avoid complicated data structures
    * SoA vs AoS
  * Do less work at run time
    * Redundant computations
    * Move work to compile time 
    
Some good news: Compilers are steadily making optimizations for us.. Many of "yesterdays" optimization topics are now done for us
    

# Using ```emplace_back``` vs ```push_back``` for vectors

In [4]:
//Lets make a simple class that has a printout when it is created

class Widget {
public:
    Widget(int in_w) { w=in_w; std::cout << "New widget1\n"; }
    Widget(const Widget&) { std::cout << "New widget2\n"; }
    ~Widget() { }
private:
    int w; 
};


In [5]:
Widget a(10);

New widget1


Now a vector of them

In [6]:
std::vector<Widget> my_widgets(5,(10));
std::cout << "And reserve more space..\n";
my_widgets.reserve(7);

New widget1
New widget2
New widget2
New widget2
New widget2
New widget2
And reserve more space..
New widget2
New widget2
New widget2
New widget2
New widget2


I can then extend the vector directly

In [7]:
my_widgets.push_back(Widget(10));

New widget1
New widget2


A temporary Widget got created, so we did 2x the work. <br> In this case ```emplace_back``` can help. The syntax is somewhat different and the arguments are interpreted as the constructor arguments

In [8]:
my_widgets.emplace_back(10);

New widget1


# Pass (and receive) by reference

Consider this function, how can its memory usage (and thus speed) be improved?

In [1]:
std::vector<float> average(std::vector< std::pair<float,float> > pairs) {
    std::cout << "pairs address in average " << &pairs <<std::endl;
    std::vector<float> ave_vector;
    ave_vector.reserve(pairs.size());
    for (const auto &pair: pairs) 
        ave_vector.push_back(0.5*(pair.first + pair.second));
    std::cout << "vector address in average" << &ave_vector <<std::endl;
    return ave_vector;
}

std::vector< std::pair<float,float> > pairs;
pairs.emplace_back(25.0, 5.0);
pairs.emplace_back(5.0, 15.0);

std::cout << "pairs address " << &pairs <<std::endl;

auto results = average(pairs);
std::cout << "vector address " << &results <<std::endl;

for (const auto &result: results) 
    std::cout << result << " ";
std::cout << std::endl;


pairs address 0x7f52380e7000
pairs address in average 0x7f521bffdc28
vector address in average0x7f52380e7020
vector address 0x7f52380e7020
15 10 


We made a copy of two expensive vectors. Passing by reference solves both of these.
  * Inputs: use ```const &``` when the function is not expected to change the input
  * Outputs: Consider passing return values as arguments when expensive

In [11]:
void average_improved(const std::vector< std::pair<float,float> > &pairs,
                      std::vector<float> &ave_vector) {
    std::cout << "pairs address in average " << &pairs <<std::endl;
    ave_vector.clear(); //protect the output against previous values if desireable
    ave_vector.reserve(pairs.size());
    for (const auto &pair: pairs) 
        ave_vector.push_back(0.5*(pair.first + pair.second));
    std::cout << "vector address in average" << &ave_vector <<std::endl;
    return ave_vector;
}

std::vector< std::pair<float,float> > pairs;
pairs.emplace_back(25.0, 5.0);
pairs.emplace_back(5.0, 15.0);
std::cout << "pairs address " << &pairs <<std::endl;

std::vector<float> results;
average_improved(pairs, results);
std::cout << "vector address " << &results <<std::endl;

for (const auto &result: results) 
    std::cout << result << " ";
std::cout << std::endl;


pairs address 0x7f39bd182000
pairs address in average 0x7f39bd182000
vector address in average0x7f39bd182020
vector address 0x7f39bd182020
15 10 


A small code change avoids both of these copies. For simple data types there is no difference because the compiler handles it for you. For complex data structures always look at what can be passed by reference and at how the interface can minimize unneeded copies

# (Improving) Cache locality

* It is important to use the cache as much as possible for better performance

<img src="./images/cache.png" width="500" />

# Goal: Neighboring memory accesses should be close together

The easiest example is a 2D array. If we loop correctly, we access one memory location after its neighbor

In [12]:
std::array< std::array<int, 4>, 5> my2d{};
for (int i=0; i< my2d.size(); i++) 
    for (int j=0; j< my2d[0].size(); j++)
        std::cout << &(my2d[i][j]) << " ";
std::cout << std::endl;



0x7f39bd17f000 0x7f39bd17f004 0x7f39bd17f008 0x7f39bd17f00c 0x7f39bd17f010 0x7f39bd17f014 0x7f39bd17f018 0x7f39bd17f01c 0x7f39bd17f020 0x7f39bd17f024 0x7f39bd17f028 0x7f39bd17f02c 0x7f39bd17f030 0x7f39bd17f034 0x7f39bd17f038 0x7f39bd17f03c 0x7f39bd17f040 0x7f39bd17f044 0x7f39bd17f048 0x7f39bd17f04c 


But if we reverse the indices, we jump around. In big matrices, this is important

In [13]:
std::array< std::array<int, 4>, 5> my2d{};
for (int i=0; i< my2d[0].size(); i++) 
    for (int j=0; j< my2d.size(); j++)
        std::cout << &(my2d[j][i]) << " ";
std::cout << std::endl;



0x7f39bd17c000 0x7f39bd17c010 0x7f39bd17c020 0x7f39bd17c030 0x7f39bd17c040 0x7f39bd17c004 0x7f39bd17c014 0x7f39bd17c024 0x7f39bd17c034 0x7f39bd17c044 0x7f39bd17c008 0x7f39bd17c018 0x7f39bd17c028 0x7f39bd17c038 0x7f39bd17c048 0x7f39bd17c00c 0x7f39bd17c01c 0x7f39bd17c02c 0x7f39bd17c03c 0x7f39bd17c04c 


# Structures of arrays

In adopting C++, HEP moved from Fortran common blocks to an object oriented class structure
    
Common blocks (as I used them to write my thesis) tended to look like
   * ```track_px[n_track]```
   * ```track_py[n_track]```
   * ```track_pz[n_track]```
   * or perhaps ```track_p[n_track,3]```

With C++ we instead do
   * ```std::vector<Track>``` 
   
Eg, we moved from "structures of arrays" to "arrays of structures". This change is costly for cache locality. In computationally intensive kernels it is important to undone this change.
  * Particuarly important for vectorized CPU kernels or GPU kernels

# Optimizing cache in general

Try to avoid:
  * Too many out-of-order jumps
  * Too many functional calls with large and many arguments
  * High level of recursion
  * Not freeing memory even after its last use
  * Random memory access

# Do less work (at run time)

Avoid redunant calculations (seems obvious, but it isn't..). Most important example: avoid recomputation inside a loop (Particularly if it include ```/```, ```pow```, trig functions, etc)
  

In [17]:
double m_higgs = 125.;
std::vector<float> nll_coeff {1.0, 0.01, 1.5e-4, 2.6e-6};
std::vector<float> result;

double sum=0;
int i=0;
for (auto c: nll_coeff ) {
    i+=1;
    sum+= c/(std::pow(m_higgs,i));
}
std::cout << "Total to all orders is " << sum << std::endl;
    

Total to all orders is 0.00800064


(The compiler might do it for you...) Optimize by moving the division outside of the loop

In [18]:
double m_higgs = 125.;
std::vector<float> nll_coeff {1.0, 0.01, 1.5e-4, 2.6e-6};
std::vector<float> result;

double sum=0;
int i=0;
double m_higgs_inv = 1.0/m_higgs;

for (auto c: nll_coeff ) {
    i+=1;
    sum+= c*(std::pow(m_higgs_inv,i));
}
std::cout << "Total to all orders is " << sum << std::endl;
    

Total to all orders is 0.00800064
